In [ ]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import google.generativeai as genai
import gradio as gr
import subprocess
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
or_api_key = os.getenv('OR_API_KEY')

In [2]:
openai = OpenAI()
genai.configure(api_key=google_api_key)
google = genai.GenerativeModel("gemini-2.5-flash")
openrouter = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=or_api_key
)

OPENAI_MODEL = "gpt-4o-mini"
GOOGLE_MODEL = "gemini-2.5-pro"
OPENROUTER_MODEL = "deepseek/deepseek-chat-v3.1:free"


In [3]:
system_message = """You are an assistant that reimplements Python code in high performance C++ for an AMD RYZEN 5 7520u in ubuntu 22 lts.
Respond only with C++ code; use comments sparingly and do not provide any explanation of any thing. If the Python code uses any libraries, use equivalent C++ libraries.
The C++ response needs to produce an identical output in the fastest possible time it run on ubuntu 22 lts machine."""

In [4]:
def user_prompt_for(python):
    user_prompt = """Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time.
Respond only with C++ code; do not explain your work or provide any commentary.
Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip,iostream,etc.
Here Below is the Python code:"""
    user_prompt += python
    return user_prompt

In [5]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [6]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [38]:
def optimize_openrouter(python):    
    stream = openrouter.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [34]:
def optimize_google(python):
    messages = messages_for(python)
    messages.append({'role':'system','parts': messages[0]['content']})
    messages.pop(0)
    messages.append({'role':'user','parts': messages[0]['content']})
    messages.pop(0)
    messages.pop(0)
    stream = google.generate_content(messages)
    reply = ""
    for chunk in stream:
        fragment = chunk.text or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [16]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, double param1, double param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4.0, 1.0) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12) << "Result: " << result << std::endl;
    std::chrono::duration<double> execution_time = end_time - start_time;
    std::cout << std::fixed << std::setprecision(6) << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [13]:
exec(pi)

Result: 3.141592658589
Execution Time: 14.959642 seconds


In [ ]:
!clang++ -O3 -std=c++20 -march=znver3 -mtune=znver3 -pipe -flto=thin -fuse-ld=lld -funroll-loops -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.534149 seconds


In [35]:
optimize_google(pi)

```cpp
#include <chrono>   // For std::chrono::high_resolution_clock
#include <cstdio>   // For printf

// The calculate function performs the core computation.
// It uses 'long long' for the iteration count to prevent potential overflow
// if 'iterations' were to exceed the capacity of a 32-bit integer.
// 'param1' and 'param2' are passed as 'double' to ensure all arithmetic
// within the loop is performed using double-precision floating-point numbers,
// matching Python's default 'float' type precision.
double calculate(long long iterations, double param1, double param2) {
    double result = 1.0; // Initialize result as a double
    
    // Loop from 1 up to 'iterations' (inclusive)
    for (long long i = 1; i <= iterations; ++i) {
        // Explicitly cast 'i' to double for floating-point multiplication.
        // This ensures that 'current_i_double * param1' is a double calculation.
        double current_i_double = static_cast<double>(i);
        
        // Calculate j_minus. 

In [37]:
!clang++ -O3 -std=c++20 -march=znver3 -mtune=znver3 -pipe -flto=thin -fuse-ld=lld -funroll-loops -o optimizedg optimized.cpp
!./optimizedg

Result: 3.141592658589
Execution Time: 0.531511 seconds


In [39]:
optimize_openrouter(pi)
!clang++ -O3 -std=c++20 -march=znver3 -mtune=znver3 -pipe -flto=thin -fuse-ld=lld -funroll-loops -o optimizedor optimized.cpp
!./optimizedor

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, double param1, double param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4.0, 1.0) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> elapsed = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;

    return 0;
}
```Result: 3.141592658589
Execution Time: 0.529022814000 seconds


In [40]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [41]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 51.393134 seconds


In [42]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>

class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1ul << 32)
        : value(seed), a(a), c(c), m(m) {}

    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    uint32_t value, a, c, m;
};

uint64_t max_subarray_sum(size_t n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    for (size_t i = 0; i < n; ++i) {
        random_numbers[i] = (lcg.next() % (max_val - min_val + 1)) + min_val;
    }

    int64_t max_sum = std::numeric_limits<int64_t>::min();
    for (size_t i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (size_t j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

uint64_t total_max_su

In [43]:
!clang++ -O3 -std=c++20 -march=znver3 -mtune=znver3 -pipe -flto=thin -fuse-ld=lld -funroll-loops -o optimizedc optimized.cpp
!./optimizedc

optimized.cpp:9:88: warning: implicit conversion from 'unsigned long' to 'uint32_t' (aka 'unsigned int') changes value from 4294967296 to 0 [-Wconstant-conversion]
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1ul << 32)
                                                                                 ~ ~~~~^~~~~
1 warning generated.
Total Maximum Subarray Sum (20 runs): 0
Execution Time: 0.753028 seconds


In [44]:
optimize_google(python_hard)

```cpp
#include <iostream>   // For std::cout, std::endl
#include <vector>     // For std::vector
#include <cstdint>    // For uint32_t, uint64_t
#include <limits>     // For std::numeric_limits
#include <chrono>     // For std::chrono::high_resolution_clock, std::chrono::duration
#include <iomanip>    // For std::fixed, std::setprecision

// LCG generator structure
struct LCG {
    uint32_t current_value;
    const uint32_t a = 1664525;
    const uint32_t c = 1013904223;
    // The modulus m = 2^32 is implicitly handled by the uint32_t type.
    // Intermediate calculation uses uint64_t to prevent overflow before the modulo.

    LCG(uint32_t seed) : current_value(seed) {}

    // Generates and returns the next pseudo-random number in the sequence.
    uint32_t next() {
        current_value = static_cast<uint32_t>(static_cast<uint64_t>(a) * current_value + c);
        return current_value;
    }
};

// Calculates the maximum subarray sum using a naive O(N^2) approach.
// Uses long lo

In [45]:
!clang++ -O3 -std=c++20 -march=znver3 -mtune=znver3 -pipe -flto=thin -fuse-ld=lld -funroll-loops -o optimizedg optimized.cpp
!./optimizedg

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.703390 seconds


In [46]:
optimize_openrouter(python_hard)
!clang++ -O3 -std=c++20 -march=znver3 -mtune=znver3 -pipe -flto=thin -fuse-ld=lld -funroll-loops -o optimizedor optimized.cpp
!./optimizedor

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>

class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1U << 32) 
        : value(seed), a(a), c(c), m(m) {}

    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    uint32_t value, a, c, m;
};

int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int64_t> random_numbers(n);
    
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }
    
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

int64_t total_max_

In [47]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [60]:
def stream_openrouter(python):    
    stream = openrouter.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [54]:
def stream_google(python_code: str):

    messages = [
        {"role": "user", "parts": user_prompt_for(python_code)}
    ]

    # Start streaming
    stream = google.generate_content(messages, stream=True)
    reply = ""

    for chunk in stream:
        if chunk.text:   # Only add non-empty parts
            reply += chunk.text
            yield reply.replace("```cpp\n", "").replace("```", "")

In [55]:
for output in stream_google(pi):
    print(output, end='\r', flush=True)

#include <iostream>   // For std::cout, std::endl
#include <iomanip>    // For std::fixed, std::setprecision
#include <iostream>   // For std::cout, std::endl
#include <iomanip>    // For std::fixed, std::setprecision
#include <chrono>     // For std::chrono::high_resolution_clock and duration

// Function to perform the calculation
// Uses long long for iterations to safely handle large numbers,
#include <iostream>   // For std::cout, std::endl small integers in the problem.
#include <iomanip>    // For std::fixed, std::setprecision
#include <chrono>     // For std::chrono::high_resolution_clock and duration

// Function to perform the calculation
// Uses long long for iterations to safely handle large numbers,
// and int for param1_int, param2_int as they are small integers in the problem.
// These integer parameters are converted to double inside the function for
// precise floating-point arithmetic.
double calculate(long long iterations, int param1_int, int param2_int) {
#include <

In [56]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Google":
        result = stream_google(python)
    elif model=="OpenRouter":
        result = stream_openrouter(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [61]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Google","OpenRouter"], label="Select model", value="OpenRouter")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [62]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [63]:
def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = "clang++ -O3 -std=c++20 -march=znver3 -mtune=znver3 -pipe -flto=thin -fuse-ld=lld -funroll-loops -o optimized optimized.cpp".split(' ')
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [64]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [68]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Google", "OpenRouter"], label="Select model", value="OpenRouter")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

Traceback (most recent call last):
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/queueing.py", line 716, in process_events
    response = await route_utils.call_process_api(
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/blocks.py", line 1793, in call_function
    prediction = await utils.async_iteration(iterator)
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/utils.py", line 754, in async_iteration
    return await anext(iterator)
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib